In [1]:
from miles_lib import *
import pandas as pd
import os

In [2]:
seed = 0

In [3]:
print(os.getcwd())

memphis_data = pd.read_csv('MemphisData.csv')
memphis_age_data = pd.read_csv('ZipCodePopAgeBreakdown.csv')
#print(memphis_data.head())

#memphis_working_data
#memphis_car_data

zip_codes = memphis_data['ZIP'].tolist()

#Get household proportion per zip
pop_density_per_household = memphis_data['household_pop_density']
print(pop_density_per_household.head())

#Get household income per zip
median_incomes = memphis_data['Median.household.income..in.US.dollars..y']
print(median_incomes.head())

#Get age distribution per zip
corresponding_age_ranges = [[0,5], [5,10], [10,15], [15,20], [20,25], [25,30], [30,35], [35,40], [40,45], [45,50], [50,55], [55,60], [60,65], [65,70], [70,75], [75,80], [80,85], [85,200]]
proportion_each_age = memphis_age_data[["Under.5", "X.5...9", "X.10...14", "X15.19", "X20.24", "X25.29", 
                                        "X30.34", "X35.39", "X40.44", "X45.49", "X50.54", "X55.59",
                                        "X60.64", "X65.69", "X70.74", "X75.79", "X80.84", "X85.Plus"]]
print(memphis_age_data.head())

#Get commute distribution per zip
proportion_each_commute = memphis_data[["proportion_wfh", "proportion_public", "proportion_drive"]]
print(proportion_each_commute.head())

#Get house type distribution per zip
proportion_each_house_type = memphis_data[["house1_proportion", "house2_proportion", "house3_proportion", "house4_proportion"]]
print(proportion_each_house_type.head())

#Get probability of working per zip and agedistr
proportion_working_agebin = memphis_data[["p_working_for_age_Under.5", "p_working_for_age_X.5...9", "p_working_for_age_X.10...14", "p_working_for_age_X15.19", "p_working_for_age_X20.24", "p_working_for_age_X25.29",
                                          "p_working_for_age_X30.34", "p_working_for_age_X35.39", "p_working_for_age_X40.44", "p_working_for_age_X45.49", "p_working_for_age_X50.54", "p_working_for_age_X55.59",
                                          "p_working_for_age_X60.64", "p_working_for_age_X65.69", "p_working_for_age_X70.74", "p_working_for_age_X75.79", "p_working_for_age_X80.84", "p_working_for_age_X85.Plus"]]
print(proportion_working_agebin.head())


#Get working travel distribution per zip

#Get household size per house type

/Users/milescaprio/Library/CloudStorage/OneDrive-Personal/Documents/M3Challenge/Repo/lib/competition
0    2.883316
1    2.410781
2    2.815614
3    2.672951
4    2.842008
Name: household_pop_density, dtype: float64
0    115478
1     75719
2    135946
3     89627
4    150847
Name: Median.household.income..in.US.dollars..y, dtype: int64
   Unnamed: 0    ZIP  Under.5  X.5...9  X.10...14  X15.19  X20.24  X25.29  \
0           1  38103      323      177        184     367    1574    2782   
1           2  38002     2570     3419       3972    3753    2179    1762   
2           3  38017      535      579        658     674     740     763   
3           4  38016     2992     2792       3075    3172    3383    3697   
4           5  38018     2193     2114       2358    2402    2550    2789   

   X30.34  X35.39  ...  X65.69  X70.74  X75.79  X80.84  X85.Plus   X  X.1  \
0    2111    1620  ...     578     382     179      77        73 NaN  NaN   
1    2437    3261  ...    1933    1568     932

In [12]:
def get_samples_for_house(zip):
    i = zip_codes.index(zip)
    print("Index:", i)
    
    house1prop = proportion_each_house_type.iloc[i, 0]
    house2prop = proportion_each_house_type.iloc[i, 1]
    house3prop = proportion_each_house_type.iloc[i, 2]
    house4prop = proportion_each_house_type.iloc[i, 3]
    #print(house1prop, house2prop, house3prop, house4prop)
    
    house_type = sample_house_type(house1prop, house2prop, house3prop, house4prop)
    print(house_type)
    
    house_size = sample_house_size(house_type)
    print("House Size")#, house_size)
    
    shade = shade_temp_amount(house_type)
    
    wfhprop = proportion_each_commute.iloc[i, 0]
    publicprop = proportion_each_commute.iloc[i, 1]
    driveprop = proportion_each_commute.iloc[i, 2]
    #print(wfhprop, publicprop, driveprop)
    
    commute_type = sample_commute_type(wfhprop, publicprop, driveprop)
    print("Household Commute Type:", commute_type)
    
    hhincome = median_incomes[i]
    print("Household Income Type:", hhincome)
    
    has_car = []
    
    
    age_categories = []
    are_working = []
    #To finish: temps based on schedule and age risk
    for j in range(house_size):
        age_category = sample_age_category(
            normalize_probabilities(proportion_each_age.iloc[i]),
            corresponding_age_ranges
        )
        #print(proportion_each_age.iloc[i])
        age_categories.append(age_category)
        cat_probability = proportion_working_agebin.iloc[i, j]
        is_working = sample_working_status(
            proportion_working_agebin.iloc[i, j],
        )
        are_working.append(is_working)
        has_car.append(commute_type[j] == "drive")
    print("Ages:", age_categories)
    print("Are Working:", are_working)
    print("House Has Car?: ", has_car)
    
    return [house_size, house_type, shade, commute_type, hhincome, age_categories, has_car, are_working]

samps = get_samples_for_house(38111)
#Bayesian bug


Index: 14
house3
House Size
Household Commute Type: car
Household Income Type: 52806
Ages: [[30, 35], [65, 70]]
Are Working: [False, False]
House Has Car?:  [False, False]


In [ ]:
print(samps)
def solve_PHOV(house_type, commute_type, hhincome, hascar, age_category, is_working, n):
    schedule_temp = schedule_temp_data_by_person(
        commute_type, 
        inside_temp_data=inside_temp_data(house_type),
    )
    temp = vectorize_min(schedule_temp, temp_outside - shade_temp_amount(house_type))
    
    heat_index_lambda = lambda t: heatIndex(t, 0)
    
    heat_index = vectorize(heat_index_lambda, temp)
    
    heat_danger = vectorize(heat_danger_continuous, heat_index)#relative_humidity))
    return age_risk(age_category) * demographic_restorative_coefficient(hhincome) * integral_of_PHOV(heat_danger)

PHOV = solve_PHOV(samps[1], samps[3], samps[4], samps[6], samps[5][0], samps[7][0])
print(PHOV)


[2, 'house3', 0, 'car', 52806, [[30, 35], [65, 70]], [False, False], [False, False]]


NameError: name 'n' is not defined

In [ ]:
def solve_HOV(house_type, commute_type, hhincome, hascar, age_category, is_working):
    x=0
    for i in range(samps[0]):
        x = x + solve_PHOV(house_type, commute_type, hhincome, hascar, age_category[0], is_working[0], i)
    return x

solve_HOV(samps[1], samps[3], samps[4], samps[6], samps[5], samps[7])

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value